# **Setup GPU**

In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

/home/catlab/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Found GPU at: /device:GPU:0


# **Importing the required packages**

In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import numpy as np
from numpy import random
from pickle import load
from numpy import array
import os
import re
import pandas as pd
import tensorflow as tf
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras import optimizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
#from keras.metrics import RootMeanSquaredError
import matplotlib.pyplot as plt
from keras.layers.advanced_activations import LeakyReLU
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

[nltk_data] Downloading package stopwords to /home/catlab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Using TensorFlow backend.


# **Building embedding for the words**

In [4]:
# embedding bin file
embed_file = "10k-sample/sim.expand.200d.vec"

#Define Hyper parameters
max_inp_len = 20000
# the dimension of vectors to be used
embed_dim = 200
rounding = 6
# filter sizes of the different conv layers 
filter_sizes = [3,4,5]
num_filters = 1
pool_size = 199
# dropout probability
drop = 0.5
batch_size = 50
learning_rate = 0.001
epochs = 30

In [5]:
#define embedding dictionary and embed matrix for the vocabulary
embeddings_dic = dict()
f = open(embed_file,encoding='utf8')
with open(embed_file, 'r', encoding='utf-8') as e_file:
  for line in e_file:
    splitlines = line.split()
    word = splitlines[0].strip()
    coefs = np.asarray(splitlines[1:], dtype='float32')
    embeddings_dic[word] = coefs

print("length of embedding dictionary",len(embeddings_dic))

length of embedding dictionary 70429


In [6]:
vocabulary_size = len(embeddings_dic.keys())
embed_token = Tokenizer()
embed_token.fit_on_texts(embeddings_dic.keys())
embedding_matrix = np.zeros((vocabulary_size, embed_dim))
for word, index in embed_token.word_index.items():
    embedding_matrix[index] = embeddings_dic.get(word)
print("embedding_matrix dimension",len(embedding_matrix),len(embedding_matrix[0]))
print("no of token in the tokenizer",len(embed_token.word_index) + 1)

embedding_matrix dimension 70429 200
no of token in the tokenizer 70429


# **Functions to pre process input and output**

In [7]:
#function to pre process the document
def process_doc(path_file,embed_token) :

  #tokenizing the words 
  with open(path_file,'r', encoding='utf-8') as tok_file :
    file_words = list(tok_file)[0].split()
    
  #removing the stop words
  stop_words = set(stopwords.words('english'))
  filtered_words = []  
  for word in file_words: 
      if word not in stop_words and word.isalpha(): 
          filtered_words.append(word)

  # applying stemming using PorterStemmer

  p_stemmer = PorterStemmer()
  stem_words=[]
  for word in filtered_words:
    stem_words.append(p_stemmer.stem(word))
    
  #tokenizing the words using the embed token
  tokens=[]
  for word in stem_words:
    try:
      tokens.append(embed_token.word_index[word])
    except:
      tokens.append(1)

  if len(tokens) < max_inp_len:
    tokens.extend([0]*(max_inp_len-len(tokens)))
  else:
    tokens = tokens[:max_inp_len]
    
  return np.array(tokens)

In [8]:
#output dataset
def output_data(company_id, out_path_file):
  with open(out_path_file,'r', encoding='utf-8') as out_file :
    for line in out_file.readlines():
      if company_id == line.split()[1]:
        return line.split()[0]
  return None

In [9]:
def pre_processing(meta_file,output_file):

  with open(meta_file,'r', encoding='utf-8') as m_file :
    
    year = meta_file.split('/')[2].split('.')[0]
    dir_path = os.path.dirname(meta_file).split('/')[0] + '/all.tok/' +year+'.tok'
    data =[]

    for line in m_file.readlines():
      inp_path_file = dir_path +'/'+ line.split()[0] + '.mda'
      
      # get input tokens from the company document
      inp_tokens = process_doc(inp_path_file,embed_token)
      
      # get output value for the company
      out_values = output_data(line.split()[0],output_file)

      #insert values into the data list
      data.append({'token':inp_tokens,'value':out_values})

  return data

# **Define the model**

In [10]:
def define_model(max_inp_len,vocabulary_size,embed_dim,filter_sizes,num_filters,pool_size,drop,learning_rate):
    
    # input and embedding matrix
    inputs = Input(shape=(max_inp_len,))
    embedding = Embedding(vocabulary_size, embed_dim, weights=[embedding_matrix],trainable = False)(inputs)
    
    custom_objects={'leaky_relu': tf.nn.leaky_relu}
    
    # channel 1 convolution and local max-pooling
    convolution_1 = Conv1D(filters=num_filters, kernel_size=filter_sizes[0], activation=custom_objects['leaky_relu'])(embedding)
    pool_1 = MaxPooling1D(pool_size=pool_size)(convolution_1)
    
    # channel 2 convolution and local max-pooling
    convolution_4 = Conv1D(filters=num_filters, kernel_size=filter_sizes[1], activation=custom_objects['leaky_relu'])(embedding)
    pool_2 = MaxPooling1D(pool_size=pool_size)(convolution_4)
    
    # channel 3 convolution and local max-pooling
    convolution_5 = Conv1D(filters=num_filters, kernel_size=filter_sizes[2], activation=custom_objects['leaky_relu'])(embedding)
    pool_3 = MaxPooling1D(pool_size=pool_size)(convolution_5)
    
    # merge and dropout
    merged = concatenate([pool_1,pool_2,pool_3],axis=1)
    drop_out = Dropout(drop)(merged)
    flat = Flatten()(drop_out)
    
    # 2 fully connected layers
    dense1 = Dense(100, activation=custom_objects['leaky_relu'])(flat)
    outputs = Dense(1, activation=custom_objects['leaky_relu'])(dense1)
    model = Model(inputs=[inputs], outputs=outputs)
    
    opt = optimizers.SGD()
    model.compile(loss='mse', optimizer=opt)
    
    return model

In [11]:
# define model
model = define_model(max_inp_len,vocabulary_size,embed_dim,filter_sizes,num_filters,pool_size,drop,learning_rate)
model.summary()

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20000)        0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20000, 200)   14085800    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 19998, 1)     601         embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 19997, 1)     801         embedding_1[0][0]                
__________________________

# **Data extraction and Fitting the model**

In [ ]:
test_loss_all_years = []
train_loss_all_years = []
val_loss_all_years = []
history_all_years = []
data = []
n_splits = 5
#for year in range(2008,2014):
year = 2008   
######## extracting text and storing it in dataframes ########      
data_train = pre_processing('10k-sample/all.meta/'+str(year-3)+'.meta.txt','10k-sample/all.logfama/'+str(year-3)+'.logfama.txt')
data_train.extend(pre_processing('10k-sample/all.meta/'+str(year-2)+'.meta.txt','10k-sample/all.logfama/'+str(year-2)+'.logfama.txt'))
data_train.extend(pre_processing('10k-sample/all.meta/'+str(year-1)+'.meta.txt','10k-sample/all.logfama/'+str(year-1)+'.logfama.txt'))
train_df = pd.DataFrame(data_train,columns=['token','value'])

data_test = pre_processing('10k-sample/all.meta/'+str(year)+'.meta.txt','10k-sample/all.logfama/'+str(year)+'.logfama.txt')
test_df = pd.DataFrame(data_test,columns=['token','value'])
data.append({'year':year,'train_df_length':len(data_train),'test_df_length':len(data_test)})

######## reshapping data to required format ########
CNN_train_input = train_df.token.values
CNN_train_output = [ float(x) for x in train_df.value.values ]
CNN_test_input = test_df.token.values
CNN_test_output = [ float(x) for x in test_df.value.values ]
CNN_train_output = np.array(CNN_train_output).reshape(len(CNN_train_output),1)
CNN_test_output = np.array(CNN_test_output).reshape(len(CNN_test_output),1)

CNN_train_input = np.stack(CNN_train_input)
CNN_test_input = np.stack(CNN_test_input)

scaler = MinMaxScaler()
CNN_train_output = np.array(CNN_train_output).reshape(len(CNN_train_output),1)
CNN_test_output = np.array(CNN_test_output).reshape(len(CNN_test_output),1)
output = np.concatenate((CNN_train_output, CNN_test_output))
output = scaler.fit_transform(output)
CNN_train_output = output[:len(CNN_train_input)]
CNN_test_output = output[-len(CNN_test_input):]

######## Kfold training and saving checkpoints ########
with tf.device('/device:GPU:0'):

    kf = KFold(n_splits=n_splits)
    history =[]
    train_loss=[]
    vald_loss=[]
    test_loss = []
    fold = 1

    for train_index, test_index in kf.split(CNN_train_input):

        checkpoint_filepath = 'Results/CNN_results_min/CheckPoints/'+str(year)+'CNN_checkpoint'+str(fold)
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                                filepath=checkpoint_filepath,
                                                save_weights_only=False,
                                                monitor='loss',
                                                mode='min',
                                                save_best_only=True)
        train_history = model.fit(
                              CNN_train_input[train_index],
                              CNN_train_output[train_index],#output
                              epochs=epochs, #epochs
                              verbose=1,
                              callbacks=[model_checkpoint_callback]
                          )
        model_best = tf.keras.models.load_model(checkpoint_filepath)
        fold+=1
        loss_T = model_best.evaluate(CNN_train_input[train_index],CNN_train_output[train_index], verbose=0)
        loss_V = model_best.evaluate(CNN_train_input[test_index],CNN_train_output[test_index], verbose=0)
        loss_test = model_best.evaluate(CNN_test_input,CNN_test_output, verbose=0)

        train_loss.append(loss_T)
        vald_loss.append(loss_V)
        history.append(train_history)
        test_loss.append(loss_test)

    test_loss_all_years.append(test_loss)
    train_loss_all_years.append(train_loss)
    val_loss_all_years.append(vald_loss)
    history_all_years.append(history)

Epoch 1/30


In [ ]:
stats_df = pd.DataFrame(data,columns=['year','train_df_length','test_df_length'])
stats_df.to_csv('Loss_values/CNN_stats_minmax.csv', header=False, index=False)

In [ ]:
import matplotlib.pylab as plt
data=[test_loss]
  
fig = plt.figure()  
# Creating axes instance 
ax = fig.add_axes([0, 0, 1, 1]) 
  
# Creating plot 
ax.boxplot(data)
years = [year for year in range(2008,2014)]
ax.set_xticklabels([year for year in range(2008,2014)]) 

# naming the y axis 
plt.ylabel('MSE Loss')
plt.title("Box plot for Test Loss")
textstr ='Test Loss for CNN : '+str(np.round(np.mean(test_loss),3))+' ('+str(np.round(np.std(test_loss),3))+')'
plt.gcf().text(0, -0.25, textstr, fontsize=14)
# show plot 
plt.savefig('Plots/block_plot_CNN_minmax.png',bbox_inches='tight')

In [ ]:
loss_data = []
train_data = []
vald_data = []
for year_loss_test,year_loss_train,year_loss_vald,year in zip(test_loss_all_years,train_loss_all_years,val_loss_all_years,years) :
    loss_data.append({'year':year,'value':year_loss_test})
    train_data.append({'year':year,'value':year_loss_train})
    vald_data.append({'year':year,'value':year_loss_vald})
    
loss_data_test_df = pd.DataFrame(loss_data,columns=['year','value'])
loss_data_test_df.to_csv('Loss_values/CNN_Loss_test_minmax.csv', header=False, index=False)

loss_data_train_df = pd.DataFrame(train_data,columns=['year','value'])
loss_data_train_df.to_csv('Loss_values/CNN_Loss_train_minmax.csv', header=False, index=False)

loss_data_vald_df = pd.DataFrame(vald_data,columns=['year','value'])
loss_data_vald_df.to_csv('Loss_values/CNN_Loss_vald_minmax.csv', header=False, index=False)